In [24]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

In [25]:
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

In [26]:
def generate_split(fake_cols, file_path):

    
    loader = LJDataLoader(data_path=file_path)
    loader.splitData()

    source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
    new_col_name = 'RandWaveFake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)
    
    source_architectures = ['ElevenLabs', 'UberDuck']
    new_col_name = 'EL_UD_Fake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)
    
    source_architectures = ['RandWaveFake', 'ElevenLabs', 'UberDuck']
    new_col_name = 'Fake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

    #data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
    data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=fake_cols)
    
    train_count = data_df[data_df['type'] == 'train'].shape[0]
    dev_count = data_df[data_df['type'] == 'dev'].shape[0]
    test_count = data_df[data_df['type'] == 'test'].shape[0]

    print(f'# of Train instances: {train_count}')
    print(f'# of Dev instances: {dev_count}')
    print(f'# of Test instances: {test_count}')
    
    return data_df

In [27]:
df = generate_split(['ElevenLabs', 'UberDuck', 'Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow'], '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv')
df

# of Train instances: 78330
# of Dev instances: 26110
# of Test instances: 26110


,type,id,architecture,path,label,multiclass_label
0,train,LJ008-0040,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ003-0001,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ033-0102,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ026-0041,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ031-0081,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
...,...,...,...,...,...,...
130545,test,LJ015-0204,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
130546,test,LJ013-0152,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
130547,test,LJ009-0181,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
130548,test,LJ002-0142,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9


In [28]:
def generate_features(data_df):
    speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
    embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
    em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()
    
    cadence_manager = CadenceModelManager(data_df)
    cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline() # Add param for load features or not
    
    smile_manager = SmileFeatureManager(data_df)
    #change number of features (feature_count=10 default)
    os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary', feature_count=20)
    os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass', feature_count=20)
    
    feature_store = {}
    feature_store['titanet'] = (em_feature_df, em_feature_cols)
    feature_store['openSmile_binary'] = (os_binary_feature_df, os_binary_feature_cols)
    feature_store['openSmile_multiclass'] = (os_multiclass_feature_df, os_multiclass_feature_cols)
    feature_store['cadence'] = (cad_feature_df, cad_feature_cols)
    
    return feature_store
    
    

In [29]:
def train_eval(feature_store, fake_cols):
    results_cols = ['feature_method', 'model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss']
    results = pd.DataFrame(columns=results_cols)
    
    models = ['logreg', 'random_forest']
    for model in models:
        for label_type in ['label', 'multiclass_label']:
            for k, v in feature_store.items():
                model_manager = ModelManager(model, v[0], v[1], merge_train_dev=True)
                model_manager.trainPredict(label_col=label_type)
                results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    
    return results
    
    

In [30]:
def run(fake_cols, metadata_path, name, data_df=None):
    if data_df is None:
        data_df = generate_split(fake_cols, metadata_path)
    feature_store = generate_features(data_df)
    results = train_eval(feature_store, fake_cols)
    results.to_csv(f'/home/ubuntu/data/results/{name}.csv', index=False)
    
    

In [8]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['ElevenLabs'], file_path, '16KHz_ElevenLabs')

# of Train instances: 15666
# of Dev instances: 5222
# of Test instances: 5222
[NeMo I 2023-06-13 15:43:48 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 15:43:48 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 15:43:48 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 15:43:49 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 15:43:49 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 15:43:49 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 15:43:49 features:267] PADDING: 16
[NeMo I 2023-06-13 15:43:50 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:25<00:00, 12.94s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[14]_stddev
2. pcm_fftMag_spectralEntropy_sma_qregerrQ
3. pcm_fftMag_spectralEntropy_sma_de_lpc1
4. pcm_fftMag_spectralEntropy_sma_upleveltime90
5. pcm_fftMag_spectralEntropy_sma_de_lpc4
6. pcm_RMSenergy_sma_de_peakMeanRel
7. pcm_fftMag_spectralEntropy_sma_lpc2
8. pcm_fftMag_spectralEntropy_sma_de_minPos
9. pcm_fftMag_spectralEntropy_sma_upleveltime75
10. pcm_fftMag_spectralEntropy_sma_range
11. pcm_fftMag_spectralEntropy_sma_upleveltime50
12. pcm_fftMag_spectralEntropy_sma_de_maxPos
13. pcm_fftMag_spectralEntropy_sma_de_lpgain
14. pcm_fftMag_spectralEntropy_sma_peakRangeRel
15. audspecRasta_lengthL1norm_sma_qregc2
16. pcm_fftMag_spectralEntropy_sma_de_range
17. pcm_fftMag_fband250-650_sma_de_peakMeanRel
18. pcm_fftMag_spectralHarmonicity_sma_de_peakMeanRel
19. pcm_fftMag_spectralEntropy_sma_lpc4
20. pcm_fftMag_spectralEntropy_sma_de_lpc0
Initializing data

...

Real
ElevenLabs

Class Accuracy
{'Real': 0.9996170049789352, 'ElevenLabs': 0.9992340099578706}

Accuracy
0.9994255074684029


[NeMo W 2023-06-13 15:48:10 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
ElevenLabs

Class Accuracy
{'Real': 1.0, 'ElevenLabs': 0.9988510149368058}

Accuracy
0.9994255074684029


[NeMo W 2023-06-13 15:48:13 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
ElevenLabs

Class Accuracy
{'Real': 0.8123324396782842, 'ElevenLabs': 0.8222903102259671}

Accuracy
0.8173113749521256


For the columns, we want:
- Which dataset are we using
- What are the fake cols
- Binary/Multi-Class
- Feature Generation Method
- Accuracy
- Loss

In [9]:
## SB_Comment - where are the final uned cadence results? I can see the params used below but dont think they're being used 
## SB_Comment - should set the window size and threshold as global params e.g. uberduck_window_size, elevenlabs_window_size, general_window_size etc 

In [10]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['UberDuck'], file_path, '16KHz_UberDuck')

# of Train instances: 15666
# of Dev instances: 5222
# of Test instances: 5222
[NeMo I 2023-06-13 15:48:14 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 15:48:14 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 15:48:14 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 15:48:14 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 15:48:14 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 15:48:14 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 15:48:14 features:267] PADDING: 16
[NeMo I 2023-06-13 15:48:15 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:26<00:00, 13.40s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[14]_meanFallingSlope
2. mfcc_sma[9]_lpgain
3. mfcc_sma_de[9]_iqr1-2
4. mfcc_sma_de[9]_meanRisingSlope
5. mfcc_sma[5]_lpc2
6. mfcc_sma[13]_lpgain
7. mfcc_sma[10]_meanPeakDist
8. mfcc_sma[8]_lpgain
9. mfcc_sma[14]_quartile2
10. mfcc_sma_de[13]_lpgain
11. mfcc_sma_de[9]_stddev
12. mfcc_sma[13]_meanFallingSlope
13. mfcc_sma[14]_amean
14. mfcc_sma_de[14]_meanRisingSlope
15. mfcc_sma[14]_quartile1
16. mfcc_sma_de[10]_meanPeakDist
17. mfcc_sma[11]_lpc2
18. mfcc_sma_de[9]_lpgain
19. mfcc_sma_de[12]_meanPeakDist
20. mfcc_sma_de[8]_lpgain
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



...

Real
UberDuck

Class Accuracy
{'Real': 0.9900421294523171, 'UberDuck': 0.9908081194944466}

Accuracy
0.9904251244733818


[NeMo W 2023-06-13 15:54:49 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
UberDuck

Class Accuracy
{'Real': 0.9888931443891229, 'UberDuck': 0.9888931443891229}

Accuracy
0.9888931443891229


[NeMo W 2023-06-13 15:54:54 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
UberDuck

Class Accuracy
{'Real': 0.5610877058598238, 'UberDuck': 0.5365760245116813}

Accuracy
0.5488318651857526


In [22]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['RandWaveFake'], file_path, '16KHz_RandWaveFake_B')

# of Train instances: 15666
# of Dev instances: 5222
# of Test instances: 5222
[NeMo I 2023-06-13 17:30:57 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 17:30:57 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 17:30:57 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 17:30:58 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 17:30:58 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 17:30:58 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 17:30:58 features:267] PADDING: 16
[NeMo I 2023-06-13 17:30:58 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 8/8 [01:41<00:00, 12.72s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. shimmerLocal_sma_quartile1
2. jitterLocal_sma_percentile1.0
3. mfcc_sma_de[7]_lpc0
4. mfcc_sma_de[6]_lpc2
5. jitterLocal_sma_quartile1
6. jitterDDP_sma_iqr1-2
7. jitterDDP_sma_flatness
8. mfcc_sma_de[14]_lpgain
9. mfcc_sma_de[8]_lpc1
10. mfcc_sma[9]_lpc0
11. mfcc_sma_de[10]_lpc0
12. mfcc_sma_de[5]_lpc2
13. mfcc_sma[12]_lpc1
14. jitterDDP_sma_percentile1.0
15. jitterLocal_sma_flatness
16. mfcc_sma_de[7]_lpc1
17. mfcc_sma_de[9]_lpc2
18. jitterDDP_sma_quartile1
19. jitterDDP_sma_quartile2
20. shimmerLocal_sma_percentile1.0
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



...

Real
RandWaveFake

Class Accuracy
{'Real': 0.9046342397548832, 'RandWaveFake': 0.86710072769054}

Accuracy
0.8858674837227116


[NeMo W 2023-06-13 17:38:41 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
RandWaveFake

Class Accuracy
{'Real': 0.8996553044810417, 'RandWaveFake': 0.8743776330907698}

Accuracy
0.8870164687859058


[NeMo W 2023-06-13 17:38:47 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
RandWaveFake

Class Accuracy
{'Real': 0.5354270394484871, 'RandWaveFake': 0.5009574875526618}

Accuracy
0.5181922635005745


In [11]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['UberDuck', 'ElevenLabs'], file_path, '16KHz_ElevenLabs_and_UberDuck')

# of Train instances: 23499
# of Dev instances: 7833
# of Test instances: 7833
[NeMo I 2023-06-13 15:54:55 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 15:54:55 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 15:54:55 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 15:54:56 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 15:54:56 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 15:54:56 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 15:54:56 features:267] PADDING: 16
[NeMo I 2023-06-13 15:54:56 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 3/3 [00:39<00:00, 13.12s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[14]_meanFallingSlope
2. mfcc_sma[9]_lpgain
3. pcm_fftMag_spectralEntropy_sma_de_lpc1
4. mfcc_sma_de[9]_meanRisingSlope
5. pcm_fftMag_spectralEntropy_sma_de_lpc4
6. mfcc_sma[13]_lpgain
7. mfcc_sma[12]_lpgain
8. mfcc_sma[14]_quartile2
9. mfcc_sma_de[13]_lpgain
10. pcm_fftMag_spectralEntropy_sma_upleveltime50
11. pcm_fftMag_spectralEntropy_sma_de_lpgain
12. mfcc_sma[14]_peakMeanRel
13. audspec_lengthL1norm_sma_minPos
14. mfcc_sma[14]_rqmean
15. mfcc_sma[14]_amean
16. pcm_RMSenergy_sma_minPos
17. mfcc_sma[14]_quartile1
18. mfcc_sma_de[10]_meanPeakDist
19. mfcc_sma[6]_lpc0
20. mfcc_sma[11]_lpc2
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[14]_meanFallingSlope
2. pcm_fftMag_spectralFlux_sma_minPos
3. pcm_fftMag_spectralEntropy_sma_qregerrQ
4. pcm_fftMag_spectralEntropy_sma_d

...

Real
UberDuck
ElevenLabs

Class Accuracy
{'Real': 0.9862121792416698, 'UberDuck': 0.9823822290310226, 'ElevenLabs': 0.9977020298736117}

Accuracy
0.9887654793821014


[NeMo W 2023-06-13 16:04:19 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
UberDuck
ElevenLabs

Class Accuracy
{'Real': 0.9628494829567216, 'UberDuck': 0.9574875526618154, 'ElevenLabs': 0.9977020298736117}

Accuracy
0.9726796884973828


[NeMo W 2023-06-13 16:04:25 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
UberDuck
ElevenLabs

Class Accuracy
{'Real': 0.42857142857142855, 'UberDuck': 0.5093833780160858, 'ElevenLabs': 0.8130984297204137}

Accuracy
0.5836844121026427


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['EL_UD_Fake'], file_path, '16KHz_Mix')

In [12]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['ElevenLabs', 'UberDuck', 'RandWaveFake'], file_path, '16KHz_Mix')

# of Train instances: 31332
# of Dev instances: 10444
# of Test instances: 10444
[NeMo I 2023-06-13 16:04:26 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 16:04:26 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 16:04:26 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 16:04:26 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 16:04:26 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 16:04:26 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 16:04:26 features:267] PADDING: 16
[NeMo I 2023-06-13 16:04:27 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 10/10 [02:19<00:00, 13.93s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[9]_lpgain
2. jitterLocal_sma_percentile1.0
3. mfcc_sma[14]_quartile3
4. pcm_fftMag_fband1000-4000_sma_quartile1
5. audspec_lengthL1norm_sma_amean
6. pcm_fftMag_spectralEntropy_sma_de_lpc1
7. audspec_lengthL1norm_sma_rqmean
8. pcm_RMSenergy_sma_de_peakMeanRel
9. mfcc_sma[13]_lpgain
10. jitterDDP_sma_flatness
11. mfcc_sma_de[14]_lpgain
12. mfcc_sma_de[13]_lpgain
13. mfcc_sma_de[6]_lpc0
14. pcm_fftMag_spectralEntropy_sma_upleveltime75
15. mfcc_sma[14]_amean
16. jitterDDP_sma_percentile1.0
17. jitterDDP_sma_quartile1
18. jitterDDP_sma_quartile2
19. mfcc_sma_de[9]_lpgain
20. mfcc_sma_de[8]_lpgain
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[9]_lpgain
2. pcm_fftMag_spectralEntropy_sma_qregerrQ
3. mfcc_sma[5]_lpc2
4. pcm_fftMag_spectralEntropy_sma_lpc2
5. mfcc_sma[13]_lpgain
6. jit

...

Real
ElevenLabs
UberDuck
RandWaveFake

Class Accuracy
{'Real': 0.8858674837227116, 'ElevenLabs': 0.9984680199157411, 'UberDuck': 0.9571045576407506, 'RandWaveFake': 0.8261202604366144}

Accuracy
0.9168900804289544


[NeMo W 2023-06-13 16:19:00 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
ElevenLabs
UberDuck
RandWaveFake

Class Accuracy
{'Real': 0.872845653006511, 'ElevenLabs': 0.9977020298736117, 'UberDuck': 0.9613175028724626, 'RandWaveFake': 0.8360781309842972}

Accuracy
0.9169858291842206
Real
ElevenLabs
UberDuck
RandWaveFake


[NeMo W 2023-06-13 16:19:09 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    



Class Accuracy
{'Real': 0.2799693603983148, 'ElevenLabs': 0.7786288778245882, 'UberDuck': 0.4128686327077748, 'RandWaveFake': 0.28456530065109154}

Accuracy
0.4390080428954424


In [31]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['EL_UD_Fake'], file_path, '16KHz_EL_UD_Fake_B')

# of Train instances: 15666
# of Dev instances: 5222
# of Test instances: 5222
[NeMo I 2023-06-15 07:13:21 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-15 07:13:21 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-15 07:13:21 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-15 07:13:22 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-15 07:13:22 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-15 07:13:22 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-15 07:13:22 features:267] PADDING: 16
[NeMo I 2023-06-15 07:13:23 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 3/3 [00:37<00:00, 12.43s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[9]_lpgain
2. mfcc_sma[12]_meanPeakDist
3. pcm_fftMag_spectralEntropy_sma_de_lpc1
4. audspec_lengthL1norm_sma_rqmean
5. mfcc_sma[5]_lpc2
6. mfcc_sma[13]_lpgain
7. mfcc_sma[10]_meanPeakDist
8. mfcc_sma[14]_quartile2
9. mfcc_sma_de[13]_lpgain
10. mfcc_sma[10]_lpc2
11. pcm_fftMag_spectralEntropy_sma_upleveltime50
12. mfcc_sma[14]_rqmean
13. mfcc_sma[14]_amean
14. pcm_fftMag_spectralEntropy_sma_kurtosis
15. mfcc_sma[3]_lpc2
16. mfcc_sma[14]_quartile1
17. pcm_fftMag_spectralEntropy_sma_de_range
18. mfcc_sma_de[10]_meanPeakDist
19. mfcc_sma[11]_lpc2
20. mfcc_sma_de[12]_meanPeakDist
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



...

Real
EL_UD_Fake


[NeMo W 2023-06-15 07:19:43 nemo_logging:349] /tmp/ipykernel_1951049/4098438169.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
EL_UD_Fake


[NeMo W 2023-06-15 07:19:47 nemo_logging:349] /tmp/ipykernel_1951049/4098438169.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
EL_UD_Fake


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
run(['Fake'], file_path, '16KHz_Fake')

In [15]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['ElevenLabs'], file_path, '16KHz_ElevenLabs_Laundered')

# of Train instances: 15664
# of Dev instances: 5222
# of Test instances: 5222
[NeMo I 2023-06-13 16:35:46 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 16:35:46 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 16:35:46 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 16:35:47 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 16:35:47 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 16:35:47 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 16:35:47 features:267] PADDING: 16
[NeMo I 2023-06-13 16:35:47 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:28<00:00, 14.20s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audSpec_Rfilt_sma[2]_linregc1
2. audspecRasta_lengthL1norm_sma_linregc1
3. audSpec_Rfilt_sma[4]_qregc2
4. pcm_RMSenergy_sma_de_lpc4
5. pcm_fftMag_spectralFlux_sma_de_peakMeanRel
6. audSpec_Rfilt_sma[4]_maxPos
7. pcm_RMSenergy_sma_de_peakMeanRel
8. pcm_fftMag_spectralSlope_sma_upleveltime90
9. audSpec_Rfilt_sma_de[1]_maxPos
10. audSpec_Rfilt_sma[3]_linregc1
11. audSpec_Rfilt_sma[1]_linregc1
12. audSpec_Rfilt_sma[4]_centroid
13. pcm_fftMag_spectralSlope_sma_peakMeanRel
14. audspec_lengthL1norm_sma_minPos
15. audSpec_Rfilt_sma[1]_centroid
16. audspecRasta_lengthL1norm_sma_qregc2
17. audSpec_Rfilt_sma[3]_qregc2
18. audSpec_Rfilt_sma[5]_qregc1
19. pcm_fftMag_spectralHarmonicity_sma_de_peakMeanRel
20. audspecRasta_lengthL1norm_sma_qregc1
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



...

Real
ElevenLabs

Class Accuracy
{'Real': 0.9421677518192263, 'ElevenLabs': 0.9551895825354271}

Accuracy
0.9486786671773267


[NeMo W 2023-06-13 16:41:46 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
ElevenLabs

Class Accuracy
{'Real': 0.9655304481041747, 'ElevenLabs': 0.938337801608579}

Accuracy
0.9519341248563769


[NeMo W 2023-06-13 16:41:49 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
ElevenLabs

Class Accuracy
{'Real': 0.7521324636226794, 'ElevenLabs': 0.7594173782019086}

Accuracy
0.7557730923694779


In [16]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['UberDuck'], file_path, '16KHz_UberDuck_Laundered')

# of Train instances: 15664
# of Dev instances: 5222
# of Test instances: 5222
[NeMo I 2023-06-13 16:41:49 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 16:41:49 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 16:41:49 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 16:41:50 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 16:41:50 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 16:41:50 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 16:41:50 features:267] PADDING: 16
[NeMo I 2023-06-13 16:41:51 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:27<00:00, 13.52s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[14]_meanFallingSlope
2. mfcc_sma[9]_lpgain
3. mfcc_sma[14]_quartile3
4. mfcc_sma[12]_meanPeakDist
5. mfcc_sma_de[9]_meanRisingSlope
6. mfcc_sma_de[9]_rqmean
7. mfcc_sma[5]_lpc2
8. mfcc_sma[14]_peakMeanAbs
9. mfcc_sma[13]_peakMeanAbs
10. mfcc_sma[13]_lpgain
11. mfcc_sma[12]_lpgain
12. mfcc_sma_de[13]_meanFallingSlope
13. mfcc_sma[8]_lpgain
14. mfcc_sma[14]_peakMeanRel
15. mfcc_sma[14]_rqmean
16. mfcc_sma[14]_amean
17. F0final_sma_pctlrange0-1
18. mfcc_sma[14]_quartile1
19. mfcc_sma[11]_lpc2
20. mfcc_sma_de[12]_meanRisingSlope
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



...

Real
UberDuck

Class Accuracy
{'Real': 0.8582918422060514, 'UberDuck': 0.8559938720796629}

Accuracy
0.8571428571428571


[NeMo W 2023-06-13 16:47:58 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
UberDuck

Class Accuracy
{'Real': 0.8567598621217924, 'UberDuck': 0.8460360015319801}

Accuracy
0.8513979318268863


[NeMo W 2023-06-13 16:48:02 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
UberDuck

Class Accuracy
{'Real': 0.5666003976143141, 'UberDuck': 0.5617750882501261}

Accuracy
0.5642052565707134


In [17]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['RandWaveFake'], file_path, '16KHz_RandWaveFake_Laundered')

# of Train instances: 15664
# of Dev instances: 5222
# of Test instances: 5222
[NeMo I 2023-06-13 16:48:02 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 16:48:02 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 16:48:02 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 16:48:03 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 16:48:03 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 16:48:03 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 16:48:03 features:267] PADDING: 16
[NeMo I 2023-06-13 16:48:03 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 8/8 [01:49<00:00, 13.74s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. shimmerLocal_sma_quartile1
2. mfcc_sma[8]_lpc1
3. jitterLocal_sma_percentile1.0
4. shimmerLocal_sma_percentile1.0
5. mfcc_sma_de[5]_lpc1
6. mfcc_sma[6]_lpc1
7. mfcc_sma_de[6]_lpc2
8. mfcc_sma[11]_lpc1
9. jitterLocal_sma_quartile1
10. jitterDDP_sma_flatness
11. mfcc_sma[8]_lpc0
12. mfcc_sma[9]_lpc0
13. jitterDDP_sma_percentile1.0
14. mfcc_sma_de[6]_lpc1
15. jitterDDP_sma_quartile1
16. jitterDDP_sma_quartile2
17. mfcc_sma[13]_lpc1
18. mfcc_sma[9]_lpc1
19. mfcc_sma_de[13]_lpc1
20. jitterDDP_sma_de_iqr2-3
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



...

Real
RandWaveFake

Class Accuracy
{'Real': 0.7973956338567598, 'RandWaveFake': 0.7633090769819992}

Accuracy
0.7803523554193795


[NeMo W 2023-06-13 16:55:50 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
RandWaveFake

Class Accuracy
{'Real': 0.783224818077365, 'RandWaveFake': 0.7583301417081578}

Accuracy
0.7707774798927614


[NeMo W 2023-06-13 16:55:55 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
RandWaveFake

Class Accuracy
{'Real': 0.5341922695738355, 'RandWaveFake': 0.48056537102473496}

Accuracy
0.5076269067266816


In [18]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['UberDuck', 'ElevenLabs'], file_path, '16KHz_ElevenLabs_and_UberDuck_Laundered')

# of Train instances: 23496
# of Dev instances: 7833
# of Test instances: 7833
[NeMo I 2023-06-13 16:55:55 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 16:55:55 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 16:55:55 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 16:55:56 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 16:55:56 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 16:55:56 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 16:55:56 features:267] PADDING: 16
[NeMo I 2023-06-13 16:55:56 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 3/3 [00:39<00:00, 13.33s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[9]_lpgain
2. mfcc_sma[14]_quartile3
3. pcm_fftMag_spectralSlope_sma_percentile99.0
4. mfcc_sma_de[9]_meanRisingSlope
5. mfcc_sma[13]_lpgain
6. pcm_fftMag_spectralSlope_sma_upleveltime90
7. mfcc_sma[13]_amean
8. mfcc_sma[10]_peakMeanMeanDist
9. mfcc_sma[14]_quartile2
10. mfcc_sma_de[13]_lpgain
11. mfcc_sma[14]_percentile1.0
12. audspecRasta_lengthL1norm_sma_de_minPos
13. mfcc_sma[14]_peakMeanRel
14. mfcc_sma[14]_rqmean
15. mfcc_sma[14]_amean
16. mfcc_sma_de[13]_meanRisingSlope
17. mfcc_sma[3]_lpc2
18. mfcc_sma[14]_quartile1
19. pcm_fftMag_spectralHarmonicity_sma_de_peakMeanRel
20. mfcc_sma[6]_lpc0
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audSpec_Rfilt_sma[1]_maxPos
2. mfcc_sma_de[14]_meanFallingSlope
3. audSpec_Rfilt_sma[2]_qregc2
4. audSpec_Rfilt_sma[5]_linregc1
5. mfcc_sma[4]_pe

...

Real
UberDuck
ElevenLabs

Class Accuracy
{'Real': 0.8326311757947147, 'UberDuck': 0.8360781309842972, 'ElevenLabs': 0.9268479509766373}

Accuracy
0.8651857525852163


[NeMo W 2023-06-13 17:05:41 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
UberDuck
ElevenLabs

Class Accuracy
{'Real': 0.8402910762160092, 'UberDuck': 0.8556108770585983, 'ElevenLabs': 0.9341248563768672}

Accuracy
0.8766756032171582


[NeMo W 2023-06-13 17:05:45 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
UberDuck
ElevenLabs

Class Accuracy
{'Real': 0.42089402310396784, 'UberDuck': 0.47540160642570284, 'ElevenLabs': 0.7187977585328579}

Accuracy
0.5375042045072318


In [19]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['ElevenLabs', 'UberDuck', 'RandWaveFake'], file_path, '16KHz_Mix_Laundered')

# of Train instances: 31328
# of Dev instances: 10444
# of Test instances: 10444
[NeMo I 2023-06-13 17:05:46 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 17:05:46 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 17:05:46 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 17:05:46 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 17:05:46 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 17:05:46 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 17:05:46 features:267] PADDING: 16
[NeMo I 2023-06-13 17:05:47 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 10/10 [02:14<00:00, 13.41s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[9]_lpgain
2. jitterLocal_sma_percentile1.0
3. mfcc_sma[14]_quartile3
4. pcm_fftMag_fband1000-4000_sma_quartile1
5. audspec_lengthL1norm_sma_amean
6. pcm_fftMag_spectralFlux_sma_pctlrange0-1
7. pcm_fftMag_spectralFlux_sma_de_rqmean
8. jitterDDP_sma_flatness
9. audspec_lengthL1norm_sma_quartile2
10. mfcc_sma[14]_quartile2
11. mfcc_sma[14]_rqmean
12. mfcc_sma[14]_amean
13. jitterDDP_sma_percentile1.0
14. mfcc_sma[14]_quartile1
15. pcm_fftMag_spectralFlux_sma_peakRangeAbs
16. mfcc_sma[6]_lpc0
17. jitterDDP_sma_quartile1
18. jitterDDP_sma_quartile2
19. mfcc_sma_de[9]_lpgain
20. mfcc_sma[7]_lpc0
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. jitterLocal_sma_percentile1.0
2. mfcc_sma[14]_quartile3
3. pcm_fftMag_spectralSlope_sma_percentile99.0
4. audSpec_Rfilt_sma[4]_qregc2
5. audspecRasta_le

...


Class Accuracy
{'Real': 0.8100344695518958, 'ElevenLabs': 0.9410187667560321, 'UberDuck': 0.920337035618537, 'RandWaveFake': 0.8119494446572194}

Accuracy
0.8708349291459211
Real
ElevenLabs
UberDuck
RandWaveFake

Class Accuracy
{'Real': 0.7024128686327078, 'ElevenLabs': 0.7702029873611643, 'UberDuck': 0.7571811566449637, 'RandWaveFake': 0.6499425507468403}

Accuracy
0.719934890846419


[NeMo W 2023-06-13 17:20:36 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    
[NeMo W 2023-06-13 17:20:54 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
ElevenLabs
UberDuck
RandWaveFake

Class Accuracy
{'Real': 0.6641133665262352, 'ElevenLabs': 0.9302949061662198, 'UberDuck': 0.8169283799310609, 'RandWaveFake': 0.6710072769054002}

Accuracy
0.7705859823822291


[NeMo W 2023-06-13 17:21:00 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
ElevenLabs
UberDuck
RandWaveFake

Class Accuracy
{'Real': 0.30808337569903405, 'ElevenLabs': 0.6710065097646469, 'UberDuck': 0.42364282090309485, 'RandWaveFake': 0.2495019920318725}

Accuracy
0.41319400730202693


In [21]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['EL_UD_Fake'], file_path, '16KHz_EL_UD_Fake_Laundered_B')

# of Train instances: 15664
# of Dev instances: 5222
# of Test instances: 5222
[NeMo I 2023-06-13 17:24:35 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 17:24:35 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 17:24:35 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 17:24:35 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 17:24:35 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 17:24:35 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 17:24:35 features:267] PADDING: 16
[NeMo I 2023-06-13 17:24:36 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 3/3 [00:38<00:00, 12.94s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[13]_quartile2
2. audSpec_Rfilt_sma[19]_centroid
3. mfcc_sma[9]_lpgain
4. audspecRasta_lengthL1norm_sma_de_maxPos
5. mfcc_sma[14]_quartile3
6. audspec_lengthL1norm_sma_quartile3
7. mfcc_sma_de[9]_meanRisingSlope
8. mfcc_sma[14]_peakMeanAbs
9. pcm_RMSenergy_sma_de_peakMeanRel
10. mfcc_sma[10]_meanPeakDist
11. mfcc_sma[11]_pctlrange0-1
12. mfcc_sma[14]_quartile2
13. mfcc_sma_de[13]_lpgain
14. audspecRasta_lengthL1norm_sma_de_minPos
15. audspec_lengthL1norm_sma_minPos
16. mfcc_sma[14]_rqmean
17. mfcc_sma[14]_amean
18. mfcc_sma_de[13]_meanRisingSlope
19. mfcc_sma[14]_quartile1
20. mfcc_sma_de[9]_lpgain
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



...

Real
EL_UD_Fake

Class Accuracy
{'Real': 0.886633473764841, 'EL_UD_Fake': 0.868632707774799}

Accuracy
0.87763309076982


[NeMo W 2023-06-13 17:30:53 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
EL_UD_Fake

Class Accuracy
{'Real': 0.9084641899655305, 'EL_UD_Fake': 0.8789735733435465}

Accuracy
0.8937188816545385


[NeMo W 2023-06-13 17:30:57 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
EL_UD_Fake

Class Accuracy
{'Real': 0.6592292089249493, 'EL_UD_Fake': 0.5811283075386919}

Accuracy
0.619874213836478


In [23]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
run(['Fake'], file_path, '16KHz_Fake_Laundered_B')

# of Train instances: 15664
# of Dev instances: 5222
# of Test instances: 5222
[NeMo I 2023-06-13 17:38:47 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-13 17:38:47 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-13 17:38:47 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-13 17:38:48 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-13 17:38:48 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-13 17:38:48 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-13 17:38:48 features:267] PADDING: 16
[NeMo I 2023-06-13 17:38:49 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 10/10 [02:08<00:00, 12.89s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[9]_lpgain
2. mfcc_sma[14]_quartile3
3. audspec_lengthL1norm_sma_amean
4. audspec_lengthL1norm_sma_quartile3
5. mfcc_sma[14]_peakMeanAbs
6. mfcc_sma[5]_lpc0
7. pcm_fftMag_fband1000-4000_sma_iqr1-3
8. mfcc_sma[13]_lpgain
9. jitterDDP_sma_flatness
10. audspec_lengthL1norm_sma_quartile2
11. mfcc_sma_de[13]_lpgain
12. audSpec_Rfilt_sma[4]_centroid
13. mfcc_sma[14]_rqmean
14. mfcc_sma[14]_amean
15. jitterDDP_sma_percentile1.0
16. audspecRasta_lengthL1norm_sma_qregc2
17. mfcc_sma[6]_lpc0
18. jitterDDP_sma_quartile1
19. mfcc_sma_de[9]_lpgain
20. pcm_fftMag_fband1000-4000_sma_iqr2-3
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



...

Real
Fake

Class Accuracy
{'Real': 0.8295672156261968, 'Fake': 0.8004595940252777}

Accuracy
0.8150134048257373


[NeMo W 2023-06-13 17:47:01 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
Fake

Class Accuracy
{'Real': 0.8498659517426274, 'Fake': 0.8161623898889314}

Accuracy
0.8330141708157794


[NeMo W 2023-06-13 17:47:04 nemo_logging:349] /tmp/ipykernel_1951049/4172940570.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model]}))
    


Real
Fake

Class Accuracy
{'Real': 0.600802407221665, 'Fake': 0.519163763066202}

Accuracy
0.5598301274044467
